In [1]:
# python libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import liana as li
import decoupler as dc
import anndata

import session_info
#from mousipy import translate
from liana.method import cellphonedb


In [2]:
from plotnine import ggplot

In [3]:
from mousipy import translate

## Loading in data 

In [4]:
atlas = sc.read_h5ad('scANVI.h5ad')

In [5]:
adata = atlas

In [6]:
humanized_adata = translate(adata)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:14<00:00, 337.74it/s]


Found direct orthologs for 4414 genes.
Found multiple orthologs for 88 genes.
Found no orthologs for 415 genes.
Found no index in biomart for 83 genes.


  0%|                                                                                            | 0/88 [00:00<?, ?it/s]/opt/conda/envs/sc-py/lib/python3.10/site-packages/mousipy/mousipy.py:191: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/conda/envs/sc-py/lib/python3.10/site-packages/mousipy/mousipy.py:191: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
/opt/conda/envs/sc-py/lib/python3.10/site-packages/mousipy/mousipy.py:191: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  1%|▉                                                      

In [7]:
adata = humanized_adata

In [8]:
adata.var_names_make_unique()

In [9]:
adata.write_h5ad("Atlashumanized.h5ad")

... storing 'name' as categorical
... storing 'original_gene_symbol' as categorical


In [10]:
adata = adata[adata.obs['DevTP'] == 'E10.5']

In [11]:
# log1p normalize the data
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

/opt/conda/envs/sc-py/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.


In [12]:
adata.write_h5ad("Atlas_E10.5_humanized.h5ad")

In [13]:
adata = humanized_adata

In [14]:
adata = adata[adata.obs['DevTP'] == 'E11.5']

In [15]:
# log1p normalize the data
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [16]:
adata.write_h5ad("Atlas_E11.5_humanized.h5ad")

In [17]:
adata = humanized_adata

In [18]:
adata = adata[adata.obs['DevTP'] == 'E12.5']

In [19]:
# log1p normalize the data
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [20]:
adata.write_h5ad("Atlas_E12.5_humanized.h5ad")

## E10.5

Do this for each timepoint

In [21]:
cpdb_file_path = 'cellphonedb.zip'
meta_file_path = 'E10.5_meta.tsv'
counts_file_path = 'Atlas_E10.5_humanized.h5ad'
degs_file_path = 'E10.5_DEGs.tsv'
output_path = 'E10.5'

In [22]:
metadata = pd.read_csv(meta_file_path, sep = '\t')
metadata.head(3)

,Cell,CellType
0,AAACCCACATAATGCC-1_12,Chondroprogenitors
1,AAACCCAGTCACAGTT-1_12,ZPA
2,AAACCCAGTTAGAAAC-1_12,Proximal limb bud mesenchyme


In [23]:
import anndata

adata = anndata.read_h5ad(counts_file_path)
adata.shape

(5826, 4588)

In [24]:
list(adata.obs.index).sort() == list(metadata['Cell']).sort()


True

In [27]:
from cellphonedb.src.core.methods import cpdb_degs_analysis_method

cpdb_results = cpdb_degs_analysis_method.call( 
    cpdb_file_path = cpdb_file_path,
    meta_file_path = meta_file_path,
    counts_file_path = counts_file_path,
    degs_file_path = degs_file_path,
    counts_data = 'hgnc_symbol',
    score_interactions = True,
    threshold = 0.1,
        
    output_path = output_path)

[ ][CORE][18/01/24-16:29:30][INFO] [Cluster DEGs Analysis] Threshold:0.1 Precision:3
Reading user files...
The following user files were loaded successfully:
/home/host_home/Documents/Tom/Atlas/data/Cell-cell_communication/Atlas_E10.5_humanized.h5ad
/home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E10.5_meta.tsv
/home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E10.5_DEGs.tsv
[ ][CORE][18/01/24-16:29:32][INFO] Running Real Analysis
[ ][CORE][18/01/24-16:29:32][INFO] Running DEGs-based Analysis
[ ][CORE][18/01/24-16:29:32][INFO] Building results


/opt/conda/envs/sc-py/lib/python3.10/site-packages/cellphonedb/src/core/methods/cpdb_degs_analysis_method.py:422: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`


[ ][CORE][18/01/24-16:29:32][INFO] Scoring interactions: Filtering genes per cell type..


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 148.65it/s]

[ ][CORE][18/01/24-16:29:32][INFO] Scoring interactions: Calculating mean expression of each gene per group/cell type..



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 497.72it/s]


[ ][CORE][18/01/24-16:29:33][INFO] Scoring interactions: Calculating scores for all interactions and cell types..


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.89it/s]


Saved deconvoluted to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E10.5/degs_analysis_deconvoluted_01_18_2024_162943.txt
Saved deconvoluted_percents to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E10.5/degs_analysis_deconvoluted_percents_01_18_2024_162943.txt
Saved means to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E10.5/degs_analysis_means_01_18_2024_162943.txt
Saved relevant_interactions to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E10.5/degs_analysis_relevant_interactions_01_18_2024_162943.txt
Saved significant_means to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E10.5/degs_analysis_significant_means_01_18_2024_162943.txt
Saved interaction_scores to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E10.5/degs_analysis_interaction_scores_01_18_2024_162943.txt


In [28]:
print(cpdb_results.keys())

dict_keys(['deconvoluted', 'deconvoluted_percents', 'means', 'relevant_interactions', 'significant_means', 'CellSign_active_interactions', 'CellSign_active_interactions_deconvoluted', 'interaction_scores'])


In [29]:
import os
import anndata as ad
import pandas as pd
import ktplotspy as kpy
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
p= kpy.plot_cpdb(
    adata = adata,
    cell_type1 = "AER|Distal limb bud mesenchyme|Intermediate limb bud mesenchyme|Proximal limb bud mesenchyme|Chondroprogenitors|ZPA|Joint precursors|Resting zone chondrocytes|Proliferative chondrocytes|Pre-hypertrophic chondrocytes",
    cell_type2 = "AER|Distal limb bud mesenchyme|Intermediate limb bud mesenchyme|Proximal limb bud mesenchyme|Chondroprogenitors|ZPA|Joint precursors|Resting zone chondrocytes|Proliferative chondrocytes|Pre-hypertrophic chondrocytes", 
    means = cpdb_results['means'],
    pvals = cpdb_results['relevant_interactions'],
    celltype_key = "CellType",
    genes = ["BMP2","BMP3","BMP4","BMP5","BMP6","BMP7","SHH","GDF5"],
    figsize = (15,10),
    title = "Cell-Cell comm",
    max_size = 6,
    highlight_size = 0.75,
    degs_analysis = True,
    standard_scale = True,
    interaction_scores = cpdb_results['interaction_scores'],
    scale_alpha_by_interaction_scores=True)
        


In [33]:
p.save('E10.5_significantmeans.svg')

Fontsize 0.00 < 1.0 pt not allowed by FreeType. Setting fontsize = 1 pt


## E11.5

Do this for each timepoint

In [34]:
cpdb_file_path = 'cellphonedb.zip'
meta_file_path = 'E11.5_meta.tsv'
counts_file_path = 'Atlas_E11.5_humanized.h5ad'
degs_file_path = 'E11.5_DEGs.tsv'
output_path = 'E11.5'

In [35]:
metadata = pd.read_csv(meta_file_path, sep = '\t')
metadata.head(3)

,Cell,CellType
0,AAACCTGAGACGCACA-1_8,Chondroprogenitors
1,AAACCTGCACAGACAG-1_8,Chondroprogenitors
2,AAACCTGCAGGCTCAC-1_8,Chondroprogenitors


In [36]:
import anndata

adata = anndata.read_h5ad(counts_file_path)
adata.shape

(8290, 4588)

In [37]:
list(adata.obs.index).sort() == list(metadata['Cell']).sort()


True

In [38]:
from cellphonedb.src.core.methods import cpdb_degs_analysis_method

cpdb_results = cpdb_degs_analysis_method.call( 
    cpdb_file_path = cpdb_file_path,
    meta_file_path = meta_file_path,
    counts_file_path = counts_file_path,
    degs_file_path = degs_file_path,
    counts_data = 'hgnc_symbol',
    score_interactions = True,
    threshold = 0.1,
        
    output_path = output_path)

[ ][CORE][18/01/24-16:31:58][INFO] [Cluster DEGs Analysis] Threshold:0.1 Precision:3
Reading user files...
The following user files were loaded successfully:
/home/host_home/Documents/Tom/Atlas/data/Cell-cell_communication/Atlas_E11.5_humanized.h5ad
/home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E11.5_meta.tsv
/home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E11.5_DEGs.tsv
[ ][CORE][18/01/24-16:31:59][INFO] Running Real Analysis
[ ][CORE][18/01/24-16:31:59][INFO] Running DEGs-based Analysis
[ ][CORE][18/01/24-16:31:59][INFO] Building results


/opt/conda/envs/sc-py/lib/python3.10/site-packages/cellphonedb/src/core/methods/cpdb_degs_analysis_method.py:422: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`


[ ][CORE][18/01/24-16:31:59][INFO] Scoring interactions: Filtering genes per cell type..


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 123.48it/s]

[ ][CORE][18/01/24-16:31:59][INFO] Scoring interactions: Calculating mean expression of each gene per group/cell type..



100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 540.46it/s]


[ ][CORE][18/01/24-16:32:00][INFO] Scoring interactions: Calculating scores for all interactions and cell types..


100%|█████████████████████████████████████████████████████████████████████████████████| 256/256 [00:24<00:00, 10.29it/s]


Saved deconvoluted to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E11.5/degs_analysis_deconvoluted_01_18_2024_163225.txt
Saved deconvoluted_percents to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E11.5/degs_analysis_deconvoluted_percents_01_18_2024_163225.txt
Saved means to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E11.5/degs_analysis_means_01_18_2024_163225.txt
Saved relevant_interactions to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E11.5/degs_analysis_relevant_interactions_01_18_2024_163225.txt
Saved significant_means to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E11.5/degs_analysis_significant_means_01_18_2024_163225.txt
Saved interaction_scores to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E11.5/degs_analysis_interaction_scores_01_18_2024_163225.txt


In [39]:
print(cpdb_results.keys())

dict_keys(['deconvoluted', 'deconvoluted_percents', 'means', 'relevant_interactions', 'significant_means', 'CellSign_active_interactions', 'CellSign_active_interactions_deconvoluted', 'interaction_scores'])


In [40]:
import os
import anndata as ad
import pandas as pd
import ktplotspy as kpy
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
p=kpy.plot_cpdb(
    adata = adata,
     cell_type1 = "AER|Distal limb bud mesenchyme|Intermediate limb bud mesenchyme|Proximal limb bud mesenchyme|Chondroprogenitors|ZPA|Joint precursors|Resting zone chondrocytes|Proliferative chondrocytes|Pre-hypertrophic chondrocytes",
    cell_type2 = "AER|Distal limb bud mesenchyme|Intermediate limb bud mesenchyme|Proximal limb bud mesenchyme|Chondroprogenitors|ZPA|Joint precursors|Resting zone chondrocytes|Proliferative chondrocytes|Pre-hypertrophic chondrocytes", 
     means = cpdb_results['means'],
    pvals = cpdb_results['relevant_interactions'],
    celltype_key = "CellType",
   genes = ["BMP2","BMP3","BMP4","BMP5","BMP6","BMP7","SHH","GDF5"],
        figsize = (25,10),

    title = "Cell-Cell comm",
    max_size = 6,
    highlight_size = 0.75,
    degs_analysis = True,
    standard_scale = True,
    interaction_scores = cpdb_results['interaction_scores'],
    scale_alpha_by_interaction_scores=True)

In [42]:
p.save('E11.5_significantmeans.svg')

Fontsize 0.00 < 1.0 pt not allowed by FreeType. Setting fontsize = 1 pt


## E12.5

Do this for each timepoint

In [43]:
cpdb_file_path = 'cellphonedb.zip'
meta_file_path = 'E12.5_meta.tsv'
counts_file_path = 'Atlas_E12.5_humanized.h5ad'
degs_file_path = 'E12.5_DEGs.tsv'
output_path = 'E12.5'

In [44]:
metadata = pd.read_csv(meta_file_path, sep = '\t')
metadata.head(3)

,Cell,CellType
0,E12.5_AAACCTGAGGCGTACA-1_2,Synovial fibroblasts
1,E12.5_AAACCTGCACTGCCAG-1_2,Intermediate limb bud mesenchyme
2,E12.5_AAACCTGCATCAGTAC-1_2,Muscle progenitors


In [45]:
import anndata

adata = anndata.read_h5ad(counts_file_path)
adata.shape

(10080, 4588)

In [46]:
list(adata.obs.index).sort() == list(metadata['Cell']).sort()


True

In [47]:
from cellphonedb.src.core.methods import cpdb_degs_analysis_method

cpdb_results = cpdb_degs_analysis_method.call( 
    cpdb_file_path = cpdb_file_path,
    meta_file_path = meta_file_path,
    counts_file_path = counts_file_path,
    degs_file_path = degs_file_path,
    counts_data = 'hgnc_symbol',
    score_interactions = True,
    threshold = 0.1,
        
    output_path = output_path)

[ ][CORE][18/01/24-16:33:06][INFO] [Cluster DEGs Analysis] Threshold:0.1 Precision:3
Reading user files...
The following user files were loaded successfully:
/home/host_home/Documents/Tom/Atlas/data/Cell-cell_communication/Atlas_E12.5_humanized.h5ad
/home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5_meta.tsv
/home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5_DEGs.tsv
[ ][CORE][18/01/24-16:33:07][INFO] Running Real Analysis
[ ][CORE][18/01/24-16:33:07][INFO] Running DEGs-based Analysis
[ ][CORE][18/01/24-16:33:07][INFO] Building results


/opt/conda/envs/sc-py/lib/python3.10/site-packages/cellphonedb/src/core/methods/cpdb_degs_analysis_method.py:422: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`


[ ][CORE][18/01/24-16:33:07][INFO] Scoring interactions: Filtering genes per cell type..


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 138.79it/s]

[ ][CORE][18/01/24-16:33:07][INFO] Scoring interactions: Calculating mean expression of each gene per group/cell type..



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 411.54it/s]


[ ][CORE][18/01/24-16:33:08][INFO] Scoring interactions: Calculating scores for all interactions and cell types..


100%|█████████████████████████████████████████████████████████████████████████████████| 400/400 [00:38<00:00, 10.43it/s]


Saved deconvoluted to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5/degs_analysis_deconvoluted_01_18_2024_163347.txt
Saved deconvoluted_percents to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5/degs_analysis_deconvoluted_percents_01_18_2024_163347.txt
Saved means to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5/degs_analysis_means_01_18_2024_163347.txt
Saved relevant_interactions to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5/degs_analysis_relevant_interactions_01_18_2024_163347.txt
Saved significant_means to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5/degs_analysis_significant_means_01_18_2024_163347.txt
Saved interaction_scores to /home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5/degs_analysis_interaction_scores_01_18_2024_163347.txt


In [48]:
print(cpdb_results.keys())

dict_keys(['deconvoluted', 'deconvoluted_percents', 'means', 'relevant_interactions', 'significant_means', 'CellSign_active_interactions', 'CellSign_active_interactions_deconvoluted', 'interaction_scores'])


In [51]:
p= kpy.plot_cpdb(
    adata = adata,
     cell_type1 = "AER|Distal limb bud mesenchyme|Intermediate limb bud mesenchyme|Proximal limb bud mesenchyme|Chondroprogenitors|ZPA|Joint precursors|Resting zone chondrocytes|Proliferative chondrocytes|Pre-hypertrophic chondrocytes",
    cell_type2 = "AER|Distal limb bud mesenchyme|Intermediate limb bud mesenchyme|Proximal limb bud mesenchyme|Chondroprogenitors|ZPA|Joint precursors|Resting zone chondrocytes|Proliferative chondrocytes|Pre-hypertrophic chondrocytes", 
     means = cpdb_results['means'],
    pvals = cpdb_results['relevant_interactions'],
    celltype_key = "CellType",
   genes = ["BMP2","BMP3","BMP4","BMP5","BMP6","BMP7","SHH","GDF5"],
        figsize = (25,10),

    title = "Cell-Cell comm",
    max_size = 6,
    highlight_size = 0.75,
    degs_analysis = True,
    standard_scale = True,
    interaction_scores = cpdb_results['interaction_scores'],
    scale_alpha_by_interaction_scores=True)

In [52]:
p.save('/home/host_home/Documents/Tom/Atlas/output/CellphoneDB/E12.5_significantmeans.svg')

Fontsize 0.00 < 1.0 pt not allowed by FreeType. Setting fontsize = 1 pt
